In [16]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models,layers


base_dir = r'C:\Users\Hp\OneDrive\Desktop\AI MODELS\CNN\archive (1)\xray_dataset_covid19'


train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')


train_datagen = ImageDataGenerator(
    rescale=1./255,  
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)


val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,                  
    target_size=(224, 224),      
    batch_size=32,               
    class_mode='binary'          
)


val_generator = val_datagen.flow_from_directory(
    validation_dir,              
    target_size=(224, 224),      
    batch_size=32,               
    class_mode='binary'          
)

model= models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(512, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
 train_generator,
 epochs=45,
 validation_data=val_generator
)
validation_results = model.evaluate(val_generator)
print("validation loss:",validation_results[0])
print("validation accuracy:",validation_results[1])

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


C:\Users\Hp\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/45


C:\Users\Hp\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.4694 - loss: 2.9416 - val_accuracy: 0.5000 - val_loss: 0.6831
Epoch 2/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - accuracy: 0.4935 - loss: 0.6906 - val_accuracy: 0.5250 - val_loss: 0.6642
Epoch 3/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 8s/step - accuracy: 0.5647 - loss: 0.6708 - val_accuracy: 0.9750 - val_loss: 0.5336
Epoch 4/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.6452 - loss: 0.5991 - val_accuracy: 0.9500 - val_loss: 0.4070
Epoch 5/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.8028 - loss: 0.4527 - val_accuracy: 0.9500 - val_loss: 0.1697
Epoch 6/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.8155 - loss: 0.4238 - val_accuracy: 0.8750 - val_loss: 0.3377
Epoch 7/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.7597 - loss: 0.5108 - val_accuracy: 0.9250 - val_loss: 0.3498
Epoch 8/45
5/5 ━━━━━━━━━━━━━━━━━━━━ 40s 6s/step - accuracy: 0.8013 - loss: 0.4524 - val_accuracy: 0.9250 - val_loss: 0.3731
Epoch 9/45
5/5 ━━━━

In [23]:
import numpy as np
from tensorflow.keras.preprocessing import image
img_path = 'b.jpeg'
img = image.load_img(img_path,target_size=(224,224))
img_array = image.img_to_array(img)
img_array= np.expand_dims(img_array, axis=0)
img_array /=255.0
prediction = model.predict(img_array)
predicted_class = int(round(prediction[0][0]))
threshold = 0.5
predicted_class = 1 if prediction[0][0] > threshold else 0
class_labels= {0:'NORMAL',1:'PNEUMONIA'}
predicted_label = class_labels[predicted_class]
print ("Predicted label:",predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
Predicted label: PNEUMONIA


In [21]:
model.save('my_xray_model.keras')

In [22]:
from tensorflow.keras.models import load_model

# Load the model from a file with .keras extension
model = load_model('my_xray_model.keras')


In [1]:
import streamlit as st
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image

# Load the trained model (make sure to save your model after training it)
# Example: model.save('xray_model.h5') after training
model_path = 'my_xray_model.keras'
model = load_model(model_path)

# Set up the Streamlit app
st.title("Pneumonia Detection from X-ray")

# Image upload widget
uploaded_file = st.file_uploader("Upload an X-ray image", type=["jpg", "png", "jpeg"])

def predict(image_path, model):
    # Load the image with target size (224x224)
    img = load_img(image_path, target_size=(224, 224))
    
    # Convert the image to a numpy array
    img_array = img_to_array(img)
    
    # Rescale the image (normalize it)
    img_array = img_array / 255.0
    
    # Expand dimensions since the model expects a batch of images
    img_array = np.expand_dims(img_array, axis=0)

    # Get the prediction from the model
    prediction = model.predict(img_array)

    # Return the prediction as "Normal" or "Pneumonia"
    return "Pneumonia" if prediction[0] > 0.5 else "Normal"

if uploaded_file is not None:
    # Save the uploaded file to the local system (optional, you can directly use PIL Image)
    img = Image.open(uploaded_file)
    
    # Display the uploaded image
    st.image(img, caption='Uploaded X-ray image', use_column_width=True)
    
    # Save the image to a temporary location
    with open("temp.jpg", "wb") as f:
        f.write(uploaded_file.getbuffer())
    
    # Predict and display the result
    result = predict("temp.jpg", model)
    st.write(f"Prediction: {result}")


2024-09-28 17:14:09.242 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 17:14:10.834 
  command:

    streamlit run C:\Users\Hp\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-28 17:14:10.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 17:14:10.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 17:14:10.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 17:14:10.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 17:14:10.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
